<p><strong><font size="6">WALOUS</font></strong></p>

<p><strong><font size="6">Postprocessing - Resampling 1m Vectorisation etc..</font></strong></p>

This python code implement the method developed by ANAGEO (ULB). 

Code developped on Linux Mint 18.1 (Ubuntu Xenial 16.04) and GRASS GIS 7.3.svn (r71315).

# Table of Contents

<div id="toc"></div>

The following cell is a Javascript section of code for building the Jupyter notebook's table of content.

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

# Define working environment

**Import libraries**

In [2]:
# Import libraries needed for setting parameters of operating system 
import os
import sys
import csv
import tempfile
import subprocess
import glob

In [3]:
## Import multiprocessing and functools libraries
import multiprocessing
from multiprocessing import Pool
from functools import partial

** Add folder with SCR provided belong to this notebook**

In [4]:
# Add local module to the path
src = os.path.abspath('../SRC')
if src not in sys.path:
    sys.path.append(src)

** Setup environment variables for TAIS DESKTOP (Linux Mint + GRASS Dev) **

Please edit the file in `../SRC/config.py`, containing the configuration parameters, according to your own computer setup. The following cell is used to run this file.



In [5]:
run ../SRC/config_postprocess.py

In [6]:
print config_parameters

{'list_tiles': '../../../Postprocess_V1/list_tiles', 'outputfolder_Logfile': '../../../Postprocess_V1/Log_file', 'permanent_mapset': 'PERMANENT', 'locationepsg': '31370', 'outputfolder': '../../../Postprocess_V1', 'outputfolder_Vecteur': '../../../Postprocess_V1/Vecteur', 'gisdb': '../../GRASSDATA', 'location': 'WALOUS_31370', 'PYTHONLIB': '/usr/bin/python2', 'njobs': 6, 'outputfolder_Raster': '../../../Postprocess_V1/Raster', 'GISBASE': '/usr/lib/grass76'}


In [7]:
# Import functions that setup the environmental variables
import environ_variables as envi

In [8]:
# Set environmental variables
envi.setup_environmental_variables() 
# Display current environment variables of your computer
envi.print_environmental_variables()

MDMSESSION = mate 	
MANDATORY_PATH = /usr/share/gconf/mate.mandatory.path 	
MATE_DESKTOP_SESSION_ID = this-is-deprecated 	
LESSOPEN = | /usr/bin/lesspipe %s 	
MDM_LANG = fr_BE.UTF-8 	
LOGNAME = tais 	
USER = tais 	
HOME = /home/tais 	
XDG_VTNR = 8 	
PATH = /usr/local/bin:/home/tais/BIN:/home/tais/bin:/home/tais/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/usr/lib/grass76/bin:/usr/lib/grass76/script:/usr/lib/grass76/lib 	
CLICOLOR = 1 	
DISPLAY = :0.0 	
SSH_AGENT_PID = 2054 	
LANG = fr_BE.UTF-8 	
TERM = xterm-color 	
SHELL = /bin/bash 	
GIS_LOCK = $$ 	
XAUTHORITY = /home/tais/.Xauthority 	
SESSION_MANAGER = local/tais-HP-Z620-Workstation:@/tmp/.ICE-unix/1983,unix/tais-HP-Z620-Workstation:/tmp/.ICE-unix/1983 	
SHLVL = 1 	
QT_LINUX_ACCESSIBILITY_ALWAYS_ON = 1 	
INSIDE_CAJA_PYTHON =  	
QT_ACCESSIBILITY = 1 	
LD_LIBRARY_PATH = :/usr/lib/grass76/lib 	
COMPIZ_CONFIG_PROFILE = mate 	
WINDOWPATH = 8 	
GTK_OVERLAY_SCROLLING = 0 	
PYTHONPATH

** GRASS GIS Python libraries **

In [9]:
# Import libraries needed to launch GRASS GIS in the jupyter notebook
import grass.script.setup as gsetup
# Import libraries needed to call GRASS using Python
import grass.script as gscript

**Import libraries**

In [10]:
# Import function that check and create folder
from mkdir import check_create_dir

In [11]:
# Import function that generate a random name in the GRASS GIS environement
from random_layer_name import random_layer_name

## Special functions

In [12]:
# Import function that check existance and create GRASS GIS database folder if needed
from grass_database import check_gisdb, check_location, check_mapset, working_mapset
# Import functions for processing time information
from processing_time import start_processing, print_processing_time

In [118]:
def launch_mapset(mapset):
    #Declare empty list that will contain the messages to return
    return_message = []
    # Check if the location exists and create it if not, with the CRS defined by the epsg code 
    return_message.append(check_location(config_parameters["gisdb"],config_parameters['location'],config_parameters["locationepsg"]))
    # Check if mapset exists
    return_message.append(check_mapset(config_parameters["gisdb"],config_parameters['location'],mapset))
    # Change the current working GRASS GIS session mapset
    return_message.append(working_mapset(config_parameters["gisdb"],config_parameters['location'],mapset))
    # Check if a DB connexion exist and create it if not exists
    gscript.run_command('db.connect', quiet=True, flags='c')
    # Return
    return return_message

In [119]:
def GetMapsetsAccess(list_mapsets):
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        # Add mapsets with input data to the GRASS GIS research path
        for mapset in list_mapsets:
            gscript.run_command('g.mapsets', quiet=True, mapset=mapset, operation="add")
        return_message = "Access to other mapset added"
    except:
        return_message += "ERROR: Add access to other Mapsets failed. Please check for problem."
    return return_message

In [120]:
def DefineComputationRegionAndMask(tile_cat,resamp_resolution='1', grow_mmu='16'):
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        return_message = "Working on tile '%s'\n"%tile_cat
        # 'Grow' parameter is used to expend the computational region ouside of the current tile to deal with bordering objects smaller than MMU
        gscript.run_command('g.region', flags='a', raster='RF_fusion_tile_%s'%tile_cat, 
                            res=resamp_resolution, grow=grow_mmu)
        # Define MASK for clipping at the end
        gscript.run_command('r.mask', overwrite=True, quiet=True, raster='RF_fusion_tile_%s'%tile_cat) 
        gscript.run_command('g.copy', overwrite=True, quiet=True, raster='MASK,MASK_tile25') #25cm resolution mask following the original limit of the tile
        gscript.run_command('r.mask', quiet=True, flags='r') #Remove mask
        # Define MASK for processing (overlapping with the neighboring tiles)
        gscript.run_command('r.buffer', overwrite=True, quiet=True, input='MASK_tile25', 
                            output='MASK_tmp', distances=grow_mmu)
        rulefile = gscript.tempfile()
        with open(rulefile, 'w') as f:
            f.write('2 = 1\n* = *') #Reclass rule
        gscript.run_command('r.reclass', overwrite=True, quiet=True, input='MASK_tmp', output='MASK_processing', rules=rulefile)
        # Print
        return_message += "--> Setting of computational region and masks succeeded."
    except:
        return_message += "ERROR: Setting of computional region and masks failed for cutline '%s'. Please check for problem."%tile_cat
    return return_message

In [121]:
def Resamp(tile_cat):
    #Resample classification 
    return_message = ''
    try:
        gscript.run_command('r.mask', overwrite=True, quiet=True, raster='MASK_processing') 
        gscript.run_command('r.resamp.stats', overwrite=True, quiet=True, input='fusion_lc', 
                            output='resamp', method='mode')
        #Ensure for CEL type by creating a new map using r.mapcalc
        gscript.mapcalc('resamp_tmp=int(resamp)', overwrite=True, quiet=True) 
        gscript.run_command('g.rename', overwrite=True, quiet=True, raster='resamp_tmp,resamp')
        #Make a copy of the resampled map, nammed ' that will be used as first tmp map for the reclassification
        gscript.run_command('g.copy', overwrite=True, quiet=True, raster='resamp,reclass_tmp')
        return_message += "--> Resampling succeeded."
    except:
        return_message += "ERROR: Resampling for tile '%s' failed. Please check for problem."%tile_cat
    return return_message   

In [122]:
def GetForestryMask(tile_cat, schrink_growth_radius, overwrite=True):
    #Create forest mask
    return_message = ''
    try:
        #Initial forest mask (class 41 and 42)
        formula = "forestmask_binary=if((resamp==41||resamp==42),1,null())"
        gscript.mapcalc(formula, overwrite=overwrite, quiet=True)
        #Moving window modal filter
        gscript.run_command('r.neighbors', overwrite=True, quiet=True, input='forestmask_binary',
                            output='forestmask_binary_neighbors', method='mode', size='5')
        gscript.run_command('g.rename', overwrite=True, quiet=True, raster='forestmask_binary_neighbors,forestmask_binary')
        #Schrinking
        gscript.run_command('r.grow', overwrite=True, quiet=True, input='forestmask_binary',
                            output='forestmask_binary_shrink', radius='-%s'%schrink_growth_radius, old='1', new='1')
        #Growing
        gscript.run_command('r.grow', overwrite=True, quiet=True, input='forestmask_binary_shrink',
                            output='forestmask_binary_grow', radius='%s'%schrink_growth_radius, old='1', new='1')
        gscript.run_command('g.remove', flags='f', quiet=True, type='raster', name='forestmask_binary_shrink')
        gscript.run_command('g.rename', overwrite=True, quiet=True, raster='forestmask_binary_grow,forestmask_binary')
        # Print
        return_message += "--> Creation of forest mask succeeded."
    except:
        return_message += "ERROR: Creation of forest mask failed for tile '%s' failed. Please check for problem."%tile_cat
    return return_message   

In [123]:
""" 
Function to generate and execute an SQL insert query based on input 'table_name','head' and 'value_dict'. 
The key of the dictionnary 'value_dict' should will be the first column of the table (unique id) and the values should
contain a list with the values (the other columns). Please be sure that len('head') == 1+len(value_dict[key])
"""
def SqlInsert(table_name, header, value_dict, overwrite):
    sql_query = gscript.tempfile()
    fsql = open(sql_query, 'w')
    fsql.write('BEGIN TRANSACTION;\n')
    if gscript.db_table_exist(table_name):
        if overwrite:
            fsql.write('DROP TABLE %s;\n' % table_name)
        else:
            gscript.fatal(_("Table %s already exists. Use 'overwrite=True' to overwrite" % table_name))
    create_statement = 'CREATE TABLE ' + table_name + ' (cat int PRIMARY KEY);\n'
    fsql.write(create_statement)
    for col in header[1:]:
        if col.split('_')[-1] == 'cat':  # Mode column should be integer
            addcol_statement = 'ALTER TABLE %s ADD COLUMN %s integer;\n' % (table_name, col)
        else: # Proportions column should be double precision
            addcol_statement = 'ALTER TABLE %s ADD COLUMN %s double precision;\n' % (table_name, col)
        fsql.write(addcol_statement)
    for key in value_dict:
            insert_statement = 'INSERT INTO %s VALUES (%s, %s);\n' % (table_name, key, ','.join([str(x) for x in value_dict[key]]))
            fsql.write(insert_statement)
    fsql.write('END TRANSACTION;')
    fsql.close()
    gscript.run_command('db.execute', input=sql_query, quiet=True)

'''
Function to compute shape statistics for each clump and import it as table in GRASS GIS
'''
def GetClumpShapeStatistics(overwrite=True):
    # Path to temporary file for 'r.object.geometry' output
    tmp_csv = "%s_robjectgeometry"%gscript.tempfile()
    # Compute shape statistics for each clump
    gscript.run_command('r.object.geometry', overwrite=True, flags='m', input='clump', output=tmp_csv)
    # Load csv content in python dictionnary
    incsv = open(tmp_csv, 'r')
    reader = csv.reader(incsv, delimiter='|')
    header = reader.next()
    value_dict = {row[0]:row[1:] for row in reader}
    incsv.close()
    # Insert SQL
    table_name = "geom"
    SqlInsert(table_name, header, value_dict, overwrite)

'''
Function to retrieve the land cover class of each clump and import it as table in GRASS GIS
'''
def GetClumpLCClass(overwrite=True):
    # Path to temporary file for 'r.object.geometry' output
    tmp_csv = "%s_runivar"%gscript.tempfile()
    # Compute r.univar to get class label of the clump
    gscript.run_command('r.univar', overwrite=True, quiet=True, flags='t', map='reclass_tmp', zones='clump', output=tmp_csv)
    # Load csv content in python dictionnary
    incsv = open(tmp_csv, 'r')
    reader = csv.reader(incsv, delimiter='|')
    reader.next() # Skip the first row (header)
    header = ["cat","label"] # Create a header with custom column name
    value_dict = {row[0]:row[4:5] for row in reader} #Take only the column corresponding to 'min'
    incsv.close()
    # Insert SQL
    table_name = "label"
    SqlInsert(table_name, header, value_dict, overwrite)
    
'''
Function to now information on each clump to know if it belong to the forestry mask
'''
def GetForestMaskInfo(overwrite=True):
    # Path to temporary file for 'r.object.geometry' output
    tmp_csv = "%s_rzonalclasses_forest"%gscript.tempfile()
    # Compute r.univar to get class label of the clump
    gscript.run_command('r.zonal.classes', flags='n', overwrite=True, quiet=True,
                        zone_map='clump', raster='forestmask_binary', statistics='mode', csvfile=tmp_csv)
    # Load csv content in python dictionnary
    incsv = open(tmp_csv, 'r')
    reader = csv.reader(incsv, delimiter='|')
    reader.next() # Skip the first row (header)
    header = ["cat","forest_mask"] # Create a header with custom column name
    #value_dict = {row[0]:row[1] if row[1]!="NULL" else row[0]:'0' for row in reader} #Take only the column corresponding to 'min'
    value_dict = {row[0]:(row[1] if row[1]!="NULL" else '0') for row in reader} #Take only the column corresponding to 'min'
    incsv.close()
    # Insert SQL
    table_name = "forestry_mask"
    SqlInsert(table_name, header, value_dict, overwrite)

'''
Function to retrieve the neighborhood of each clump and import it as table in GRASS GIS
'''
def GetClumpNeighbor(overwrite=True):
    # Path to temporary file for 'r.object.geometry' output
    tmp_csv = "%s_neighbor"%gscript.tempfile()
    # Compute neighborhood matrix
    gscript.run_command('r.neighborhoodmatrix', overwrite=True, flags='l', input='clump', output=tmp_csv)
    # Load csv content in python dictionnary
    incsv = open(tmp_csv, 'r')
    reader = csv.reader(incsv, delimiter='|')
    header = ["cat","sega","segb","border_count"] # Create a header
    value_dict = {id_:row[:] for id_,row in enumerate(reader)} #Take only the column corresponding to 'min'
    incsv.close()
    # Insert SQL
    table_name = "rmatrix"
    SqlInsert(table_name, header, value_dict, overwrite)

""" 
Function to pivot rmatrix table to horizontal table with proportion to each class
"""
def GetNeighborClassProportion(overwrite=True):
    global labels_list
    ### Table 'tmp_a'
    table_name = "tmp_a"
    sql_query = gscript.tempfile()
    fsql = open(sql_query, 'w')
    fsql.write('BEGIN TRANSACTION;\n')
    if gscript.db_table_exist(table_name):
        if overwrite:
            fsql.write('DROP TABLE %s;\n'%table_name)
        else:
            gscript.fatal(_("Table %s already exists. Use 'overwrite=True' to overwrite"%table_name))
    create_statement = 'CREATE TABLE %s AS '%table_name
    create_statement += 'SELECT a.sega, b.label, border_count FROM rmatrix AS a '
    create_statement += 'LEFT JOIN label AS b ON a.segb=b.cat;\n'
    fsql.write(create_statement)
    fsql.write('END TRANSACTION;')
    fsql.close()
    gscript.run_command('db.execute', input=sql_query, quiet=True)
    
    ### Table 'tmp_b'
    table_name = "tmp_b"
    sql_query = gscript.tempfile()
    fsql = open(sql_query, 'w')
    fsql.write('BEGIN TRANSACTION;\n')
    if gscript.db_table_exist(table_name):
        if overwrite:
            fsql.write('DROP TABLE %s;\n'%table_name)
        else:
            gscript.fatal(_("Table %s already exists. Use 'overwrite=True' to overwrite"%table_name))
    create_statement = 'CREATE TABLE %s AS WITH '%table_name
    create_statement += 'border_lenght AS (SELECT sega, sum(border_count) AS sum_border FROM tmp_a GROUP BY sega),'
    create_statement += 'tempotable AS (SELECT a.sega, a.label, round((a.border_count*1.0/b.sum_border*1.0),8) AS percent_border FROM tmp_a AS a LEFT JOIN border_lenght AS b ON a.sega=b.sega)'
    create_statement += 'SELECT sega AS seg, label, sum(percent_border) AS percent_border FROM tempotable GROUP BY sega, label ORDER BY sega, label;\n'
    create_statement += 'UPDATE %s SET percent_border=1.0 WHERE percent_border>0.9999;\n'%table_name
    create_statement += 'UPDATE %s SET percent_border=0.0 WHERE percent_border<0.0001;\n'%table_name
    fsql.write(create_statement)
    fsql.write('END TRANSACTION;')
    fsql.close()
    gscript.run_command('db.execute', input=sql_query, quiet=True)
    
    ### Table 'pivot_prop_label'
    table_name = "pivot_prop_label"
    # Get distinct values for 'label' class
    distinctlabelquery="SELECT DISTINCT label FROM tmp_b ORDER BY 1"
#    labels_list = [x[0] for x in gscript.db_select(distinctlabelquery)] #According to classes that exist in the table
    labels_list = ['2','3','5','11','12','41','42','61','62'] #Forcing according to classes that should be included in the table
    # Declaration of colums for the pivot table
    columns=["seg INTEGER",]
    [columns.append("prop_%s NUMERIC"%label) for label in labels_list]
    # Crosstab query argument
    crosstabquery = "SELECT seg, label, percent_border FROM tmp_b ORDER  BY 1,2"
    # Built complete sql query
    sql_query = gscript.tempfile()
    fsql = open(sql_query, 'w')
    fsql.write('BEGIN TRANSACTION;\n')
    if gscript.db_table_exist(table_name):
        if overwrite:
            fsql.write('DROP TABLE %s;\n'%table_name)
        else:
            gscript.fatal(_("Table %s already exists. Use 'overwrite=True' to overwrite"%table_name))
    # Create statement for the pivot table
    pivot_statement = "SELECT seg, "
    pivot_statement += ', '.join(["SUM(CASE WHEN label = '%s' THEN percent_border END) AS prop_%s"%(cl,cl) for cl in labels_list])
    pivot_statement += " FROM tmp_b GROUP BY seg"
    create_statement = "CREATE TABLE %s AS %s;\n"%(table_name,pivot_statement)
    fsql.write(create_statement)
    # List of update queries to update each column
    [fsql.write("UPDATE %s SET prop_%s=0.0 WHERE prop_%s is null;\n"%(table_name,label,label)) for label in labels_list]
    fsql.write('END TRANSACTION;')
    fsql.close()
    gscript.run_command('db.execute', input=sql_query, quiet=True)

""" 
Function to 
"""

def AddLabelNthNeighbor(tile_cat, overwrite=True):
    # The trick was found here: https://stackoverflow.com/questions/8436919/second-maximum-and-minimum-values
    # To be able to run the command directly in GRASS, be sure the version of SQlite used in compilation is >3.25
    # The function used here is available only for recent version https://stackoverflow.com/questions/50332436/syntax-error-when-using-row-number-in-sqlite3r
    # Get the largest, second largest and third largest value
    ### SQL query
    table_name = "rank_prop"
    sql_query = gscript.tempfile()
    fsql = open(sql_query, 'w')
    fsql.write('BEGIN TRANSACTION;\n')
    if gscript.db_table_exist('tmp_c'):
        if overwrite:
            fsql.write("DROP TABLE tmp_c;\n")
        else:
            gscript.fatal(_("Table 'tmp_c' already exists. Use 'overwrite=True' to overwrite"))
    subquery = "SELECT seg, percent_border, row_number() OVER ( "
    subquery += "PARTITION BY seg order by percent_border DESC) as rnDesc FROM tmp_B"
    create_statement = "CREATE TABLE tmp_c AS "
    create_statement += 'SELECT seg, '
    create_statement += 'MAX(CASE when rnDesc = 1 THEN percent_border END) as first_prop, '
    create_statement += 'MAX(CASE when rnDesc = 2 THEN percent_border END) as second_prop, '
    create_statement += 'MAX(CASE when rnDesc = 3 THEN percent_border END) as third_prop '
    create_statement += 'from(%s) as SubQueryAlias GROUP BY seg;\n'%subquery
    fsql.write(create_statement)

    if gscript.db_table_exist(table_name):
        if overwrite:
            fsql.write("DROP TABLE %s;\n"%table_name)
        else:
            gscript.fatal(_("Table '%s' already exists. Use 'overwrite=True' to overwrite"%table_name))
    create_statement = "CREATE TABLE %s AS "%table_name
    create_statement += 'SELECT a.*,b.first_prop,b.second_prop,b.third_prop FROM pivot_prop_label AS a LEFT JOIN tmp_C AS b ON a.seg=b.seg;\n'
    fsql.write(create_statement)
    alter_statement = "ALTER TABLE %s ADD COLUMN first_label integer;\n"%table_name
    alter_statement += "ALTER TABLE %s ADD COLUMN second_label integer;\n"%table_name
    alter_statement += "ALTER TABLE %s ADD COLUMN third_label integer;\n"%table_name
    fsql.write(alter_statement)   
    update_queries = []
    for label in labels_list:
        update_queries.append("UPDATE {t} SET first_label={l} WHERE first_prop=prop_{l}".format(t=table_name, l=label))
        update_queries.append("UPDATE {t} SET second_label={l} WHERE second_prop=prop_{l}".format(t=table_name, l=label))
        update_queries.append("UPDATE {t} SET third_label={l} WHERE third_prop=prop_{l}".format(t=table_name, l=label))
    fsql.write(";\n".join(update_queries))
    fsql.write(";\n")
    fsql.write('END TRANSACTION;')
    fsql.close()    
    # Bash file
    bash_sqlite = gscript.tempfile()
    bash = open(bash_sqlite, 'w')
    bash.write('sqlite3 ../../GRASSDATA/WALOUS_31370/%s/sqlite/sqlite.db < %s'%(tile_cat,sql_query))
    bash.close() 
    try:
        subprocess.check_call(['bash', bash_sqlite], stderr=subprocess.STDOUT, )
    except subprocess.CalledProcessError:
        message =  "There was an error in the execution of the bash script.\nPlease check and fix."

""" 
Function to join all information together
"""
def GetFinalJoinedTable(overwrite=True):
    ### Table 'pivot_final'
    table_name = "pivot_final"
    sql_query = gscript.tempfile()
    fsql = open(sql_query, 'w')
    fsql.write('BEGIN TRANSACTION;\n')
    if gscript.db_table_exist(table_name):
        if overwrite:
            fsql.write('DROP TABLE %s;\n'%table_name)
        else:
            gscript.fatal(_("Table %s already exists. Use 'overwrite=True' to overwrite"%table_name))
    create_statement = 'CREATE TABLE %s AS '%table_name
    create_statement += 'SELECT a.cat AS seg, a.label, g.area, f.forest_mask, p.* FROM label AS a '
    create_statement += 'LEFT JOIN geom AS g ON a.cat=g.cat '
    create_statement += 'LEFT JOIN forestry_mask AS f ON a.cat=f.cat '
    create_statement += 'LEFT JOIN rank_prop AS p ON a.cat=p.seg ;\n'
    fsql.write(create_statement)
    fsql.write("ALTER TABLE %s ADD COLUMN reclass integer;\n"%table_name)   
    fsql.write('END TRANSACTION;')
    fsql.close()
    gscript.run_command('db.execute', input=sql_query, quiet=True)

""" 
Function to remove all intermediate tables
"""
def RemoveIntermediateTables(overwrite=True):
    list_tables_to_remove = ['label','forestry_mask','geom','rmatrix','tmp_a','tmp_b','tmp_c','pivot_prop_label','rank_prop']
    sql_query = gscript.tempfile()
    fsql = open(sql_query, 'w')
    fsql.write('BEGIN TRANSACTION;\n')
    for table_name in list_tables_to_remove:
        if gscript.db_table_exist(table_name):
            if overwrite:
                fsql.write('DROP TABLE %s;\n'%table_name)
            else:
                gscript.fatal(_("Table %s already exists. Use 'overwrite=True' to overwrite"%table_name))
    fsql.write('END TRANSACTION;')
    fsql.close()
    gscript.run_command('db.execute', input=sql_query, quiet=True)
    
""" 
Wrapper function that create clump and compute neighborhood information for reclassification using rules
"""
def GetNeighborStat(tile_cat, overwrite=True):
    # Define region based on 1m resampled LC
    gscript.run_command('g.region', raster='reclass_tmp')
    # Clump to get unique ID for each LC patch
    gscript.run_command('r.clump', overwrite=True, quiet=True,
                        flags='d', input='reclass_tmp', output='clump')  # flag '-d' to clump also diagonal cells
    GetClumpShapeStatistics(overwrite)
    GetClumpLCClass(overwrite)
    GetForestMaskInfo(overwrite=True)
    GetClumpNeighbor(overwrite)
    GetNeighborClassProportion(overwrite)
    AddLabelNthNeighbor(tile_cat, overwrite)
    GetFinalJoinedTable(overwrite)
    RemoveIntermediateTables(overwrite)

In [124]:
def ReclassAccordingToRule(tile_cat, list_case_statement, intermediate=True, overwrite=True):
    #Reclassify clump layer according to CASE statements 
    return_message = ''    
    try:
        #Recompute clumps and neighborhood information
        GetNeighborStat(tile_cat, overwrite)
        #Update SQLite table with reclass column 
        return_message = ''
        table_name = "pivot_final"
        if not gscript.db_table_exist(table_name):
            gscript.fatal(_("Table '%s' already exists. Use 'overwrite=True' to overwrite"%table_name))
        sql_query = gscript.tempfile()
        fsql = open(sql_query, 'w')
        fsql.write('BEGIN TRANSACTION;\n')
        fsql.write("UPDATE {t} SET reclass = NULL;\n".format(t=table_name))
        for case_statement in list_case_statement:
            fsql.write("UPDATE {t} SET reclass = ({case});\n".format(t=table_name, case=case_statement))
        fsql.write("UPDATE {t} SET reclass = label WHERE reclass IS NULL;\n".format(t=table_name))
        fsql.write('END TRANSACTION;')
        fsql.close() 
        gscript.run_command('db.execute', input=sql_query, quiet=True)
        #Create r.reclass rule file and reclass the clump map.
        query = "SELECT seg, reclass FROM pivot_final"
        reclass_rule = gscript.tempfile()
        reclass = open(reclass_rule, 'w')
        [reclass.write("%s = %s\n"%(x[0],x[1])) for x in gscript.db_select(query)]
        reclass.close()
        #r.reclass
        if intermediate:
            output_name = 'reclass_tmp'
        else:
            output_name = 'resamp_mmu'
        gscript.run_command('r.reclass', overwrite=True, input='clump', output=output_name, rules=reclass_rule)
        #Create a 'hard' copy of the reclassed raster
        gscript.mapcalc('%s_tmp=%s'%(output_name,output_name),overwrite=True,quiet=True)
        gscript.run_command('g.rename',overwrite=True,quiet=True,raster='%s_tmp,%s'%(output_name,output_name))
        # Print
        return_message += "--> Reclassification of clumps succeeded."
    except:
        return_message += "ERROR: Reclassification of clumps failed for cutline '%s'. Please check for problem."%tile_cat
    return return_message    

In [125]:
def ClipToTileBorder(tile_cat, rasterlist):
    #Clipping raster to match tile's border 
    return_message = ''
    for rastername in rasterlist:
        try:
            gscript.run_command('r.mask', overwrite=True, quiet=True, raster='MASK_tile25') 
            gscript.run_command('r.clip', overwrite=True, quiet=True,
                                input='%s'%rastername, output='tmp_%s'%rastername)
            gscript.run_command('r.mask', quiet=True, flags='r') 
            gscript.run_command('g.rename', overwrite=True, quiet=True, raster='tmp_%s,%s'%(rastername,rastername))
            return_message += "--> Clip raster '%s' to correspond to tile border."%rastername
        except:
            return_message += "ERROR: Clipping raster '%s' to tile border failed for cutline '%s'. Please check for problem."%(rastername,tile_cat)
        if rastername != rasterlist[-1]: return_message += "\n"
    return return_message   

In [126]:
def ColorizeRaster(tile_cat, rasterlist):
    #Declare empty string that will contain the messages to return
    return_message = ''
    for rastername in rasterlist:
        try:
            gscript.run_command('r.colors', map=rastername, rules=data['color_file'])
            return_message += "--> Application of colors table succeeded."
        except:
            return_message += "ERROR: Application of colors table on raster failed for tile %s. Please check for problem."%tile_cat
        if rastername != rasterlist[-1]: return_message += "\n"
    return return_message

In [127]:
def Vectorize(tile_cat):
    #Resample classification 
    return_message = ''
    try:
        gscript.run_command('r.to.vect', flags='s', overwrite=True, quiet=True,
                            input='resamp_mmu', output='vect', type='area', column='class')
        gscript.run_command('r.to.vect', overwrite=True, quiet=True,
                            input='resamp_mmu', output='vect_staircase', type='area', column='class')
        return_message += "--> Vectorization succeeded."
    except:
        return_message += "ERROR: Vectorization of raster map failed for cutline '%s'. Please check for problem."%tile_cat
    return return_message   

In [128]:
def VectorSmooth(tile_cat, thresh_value="1000000", iter_value="2"):
    #Resample classification 
    return_message = ''
    try:
        gscript.run_command('v.generalize', overwrite=True, quiet=True, input='vect', type='area', output='vect_smooth', 
                            method='chaiken', threshold=thresh_value, iterations=iter_value)
        return_message += "--> Smoothing of vector layer succeeded."
    except:
        return_message += "ERROR: Smoothing of vector layer failed for cutline '%s'. Please check for problem."%tile_cat
    return return_message   

In [129]:
def ColorizeVector(tile_cat):
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        gscript.run_command('v.colors', map='vect', use='attr',
                            column='class', rules=data['color_file'])
        gscript.run_command('v.colors', map='vect_staircase', use='attr',
                            column='class', rules=data['color_file'])
        gscript.run_command('v.colors', map='vect_smooth', use='attr',
                            column='class', rules=data['color_file'])
        return_message += "--> Application of colors table for vector layer succeeded."
    except:
        return_message += "ERROR: Application of colors table on vector failed for tile %s. Please check for problem."%tile_cat
    return return_message

In [130]:
def ExportRast(tile_cat, rasterlist):
    global list_rast_fusion
    #Declare empty string that will contain the messages to return
    return_message = ''
    for rastername in rasterlist:
        try:
            # Export raster
            export_dir = os.path.join(config_parameters['outputfolder_Raster'],rastername)
            check_create_dir(export_dir, quiet=True)
            export_path = os.path.join(export_dir,"LC_%s_tile_%s.tif"%(rastername,tile_cat))
            gscript.run_command('r.out.gdal', quiet=True, overwrite=True, input=rastername, output=export_path,
                                format='GTiff', createopt='COMPRESS=DEFLATE') #Use flag c to not export colortable. Flag m to not export non-standard format of meta-data
            return_message += "--> Export of raster '%s' suceeded."%rastername
        except:
            return_message += "ERROR: Export of raster '%s' failed for cutline '%s'. Please check for problem."%(rastername,tile_cat)
        if rastername != rasterlist[-1]: return_message += "\n"
    return return_message

In [131]:
def ExportVect(tile_cat, vectlist):
    global list_rast_fusion
    #Declare empty string that will contain the messages to return
    return_message = ''
    for vectname in vectlist:
        try:
            # Export vector
            export_dir = os.path.join(config_parameters['outputfolder_Vecteur'],vectname)
            check_create_dir(export_dir, quiet=True)
            export_path = os.path.join(export_dir, "LC_%s_tile_%s.gpkg"%(vectname,tile_cat))
            gscript.run_command('v.out.ogr', quiet=True, overwrite=True,  input=vectname,
                                output=export_path, format='GPKG') #Use flag c to not export colortable. Flag m to not export non-standard format of meta-data
            return_message += "--> Export of vector suceeded."
        except:
            return_message += "ERROR: Export of vector failed for cutline '%s'. Please check for problem."%tile_cat
    return return_message

In [132]:
def Clean(tile_cat):
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        for layer in list_rast_fusion:
            gscript.run_command('g.remove', quiet=True, flags='f', type="raster", name=layer)
        return_message += "--> Mapset cleaned"
    except:
        return_message += "ERROR: during mapset cleaning. Please check for problem."
    return return_message

In [141]:
def Worker(tile_cat):
    import subprocess
    start_tile = start_processing() 
#    print "Start processing on tile %s"%tile_cat
    #Declare empty list for saving output messages
    output_message = [] 
    
    # Launch mapset
    message = launch_mapset(tile_cat)  
    [output_message.append(a) for a in message]
#    print "\n".join(message)

    # Allow access to other mapset 
    message = GetMapsetsAccess(["FUSIONS"])
    output_message.append(message)
#    print message
    
    # Define computional region and mask
    message = DefineComputationRegionAndMask(tile_cat,resamp_resolution='1', grow_mmu='16')
    output_message.append(message)
#    print message    

    # Resample
    message = Resamp(tile_cat)
    output_message.append(message)
#    print message 
    
    # Create forestry mask
    message = GetForestryMask(tile_cat, schrink_growth_radius='15', overwrite=True)
    output_message.append(message)
#    print message 

    # Reclassification (removing MMU) according to rules 
    # MMU of 15m everywhere with special rule to prevent growing of buildings from small patches sharing the largest border with a building
    message = ReclassAccordingToRule(tile_cat,
        ["CASE WHEN area < 15 THEN CASE WHEN first_label = 12 THEN second_label ELSE first_label END END",],
        overwrite=True, intermediate=True)
    output_message.append(message)
#    print message 

    # Reclassification (removing MMU) according to rules
    # MMU of 150m in forestry mask
    message = ReclassAccordingToRule(tile_cat,
        ["CASE WHEN forest_mask = 1 AND area < 500 THEN CASE WHEN label in (41,42) THEN first_label ELSE label END END",],
        overwrite=True, intermediate=False)
    output_message.append(message)
#    print message 

    # Clip raster to match the extend of the initial tiles
    message = ClipToTileBorder(tile_cat, ['reclass_tmp','forestmask_binary','resamp','resamp_mmu'])
    output_message.append(message)
#    print message 

    # ColorizeRaster
    message = ColorizeRaster(tile_cat, ['reclass_tmp','resamp','resamp_mmu'])
    output_message.append(message)
#    print message
    
    # ExportRast 
    message = ExportRast(tile_cat, ['reclass_tmp','forestmask_binary','resamp','resamp_mmu'])
    output_message.append(message)
#    print message

    # Vectorize
    message = Vectorize(tile_cat)
    output_message.append(message)
#    print message 

    # Vector - Smoothing
    message = VectorSmooth(tile_cat, thresh_value="10000000", iter_value="3")
    output_message.append(message)
#    print message 

    # Vector - Generalization
    #message = VectorSmooth()
    #output_message.append(message)
#    print message 

    # ColorizeVector
    message = ColorizeVector(tile_cat)
    output_message.append(message)
#    print message 

    # ExportVect 
    message = ExportVect(tile_cat, ['vect','vect_smooth','vect_staircase'])
    output_message.append(message)
#    print message

    # Clean 
    #message = Clean(tile_cat)
    #output_message.append(message)
#    print message

    #Print processing time
    message = print_processing_time(start_tile, "Prediction for tile '%s' achieved in "%tile_cat)
    output_message.append(message)
#    print message
    
    #Export Log file
    fout = open(os.path.join(config_parameters['outputfolder_Logfile'],"Log_Prediction_tile_%s.txt"%tile_cat),"w")
    [fout.writelines('%s\n'%content) for content in output_message]
    fout.close()

## Create new directories

In [142]:
# Check and create folder if needed
check_create_dir(config_parameters['outputfolder'])
check_create_dir(config_parameters['outputfolder_Logfile'])
check_create_dir(config_parameters['outputfolder_Raster'])
check_create_dir(config_parameters['outputfolder_Vecteur'])

The folder '../../../Postprocess_V1' already exists
The folder '../../../Postprocess_V1/Log_file' already exists
The folder '../../../Postprocess_V1/Raster' has been created
The folder '../../../Postprocess_V1/Vecteur' has been created


# Import fusions

In [31]:
run ../SRC/config_postprocess.py

In [128]:
# Create mapset 
launch_mapset("FUSIONS")

["Location 'WALOUS_31370' already exist",
 "'FUSIONS' mapset already exists in location 'WALOUS_31370'",
 "You are now working in mapset 'WALOUS_31370/FUSIONS'"]

In [129]:
# Create a list of paths to files with a specific name prefix
list_file = glob.glob(os.path.join(data['fusion_folder'],"RF_fusion_tile_*.tif"))
print "There are %s paths in the list"%len(list_file)

There are 375 paths in the list


In [130]:
# Get list of cat that are included in the shapefile with tiles to be processed
import geopandas as gpd
gdf = gpd.read_file(data['tiles'][1])
tiles_aoi = list(gdf.cat)
gdf = None

In [34]:
# Keep only tiles number that are both exists in the folder containing all LC .tif files and included in the AOI to be processed
list_file = [(os.path.splitext(os.path.split(x)[-1])[0],x) for x in list_file if int(os.path.splitext(os.path.split(x)[-1])[0].split("_")[-1]) in tiles_aoi]
# Create list of cat to be processed
list_cat = [x[0].split("_")[-1] for x in list_file]
print "%s TIF files will be imported (contained in the AOI to be processed)"%len(list_file)

88 TIF files will be imported (contained in the AOI to be processed)


# Apply functions on each tile

In [134]:
# Set number of cores to use
ncores = 20

In [135]:
list_cat = ['3932','3933','5293','5294','6325','6326']

In [145]:
# Launch processes in parallel
start_parallel = start_processing()
p = Pool(ncores)
output = p.map(Worker, list_cat[:])  # Launch the processes for as many items in the list (if function with a return, the returned results are ordered thanks to 'map' function)
p.close()
p.join()
# Print
print_processing_time(start_parallel, "Computation (on %s cores) achieved in "%ncores)

'Computation (on 20 cores) achieved in 1 minutes and 33.7 seconds'

**Check log file for ERRORS**

In [143]:
# Get list of csv with classification feature of individual tiles
import glob
list_log = glob.glob(os.path.join(config_parameters['outputfolder_Logfile'],"Log_Prediction_tile_*.txt"))
print "%s log files in the folder"%len(list_log)

6 log files in the folder


In [144]:
# Declare new counter
count = 0
# Declare new list that will contain list of tile with error
tile_error_list = []
# Loop on list of log file
for logfile in list_log:
    got_error = False
    tile_num = os.path.splitext(os.path.basename(logfile))[0].split("_")[-1]
    fin = open(logfile, 'r')
    for row in fin:
        if row[:5] == "ERROR":  # If at least one line have error message, the whole file will be counted as 1 error
            got_error = True
    if got_error:    
        count += 1
        tile_error_list.append(tile_num)  # Add tile number to the list
# Print
print "%s tile(s) faced an ERROR during the processing.\n"%count

# Update tile list with only tiles that have ERROR in log 
print "\n".join(["Error on tile %s"%(a) for a in tile_error_list])

0 tile(s) faced an ERROR during the processing.




**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

# Create VRT with all raster products

In [74]:
# Create mapset 
launch_mapset("POSTPROCESS")

["Location 'WALOUS_31370' already exist",
 "'POSTPROCESS' mapset already exists in location 'WALOUS_31370'",
 "You are now working in mapset 'WALOUS_31370/POSTPROCESS'"]

In [80]:
# Add access to other mapsets
GetMapsetsAccess(["%s"%cat for cat in list_cat])

'Access to other mapset added'

In [86]:
# Generate VRT
gscript.run_command('r.buildvrt', overwrite=True, 
                    input=",".join(["forestmask_binary@%s"%cat for cat in list_cat]), 
                    output='forestmask_binary_vrt')

0

In [87]:
# Generate VRT
gscript.run_command('r.buildvrt', overwrite=True, 
                    input=",".join(["resamp@%s"%cat for cat in list_cat]), 
                    output='resamp_vrt')

0

In [90]:
# Generate VRT
gscript.run_command('r.buildvrt', overwrite=True, 
                    input=",".join(["reclass_tmp@%s"%cat for cat in list_cat]), 
                    output='reclass_tmp_vrt')

0

In [88]:
# Generate VRT
gscript.run_command('r.buildvrt', overwrite=True, 
                    input=",".join(["resamp_mmu@%s"%cat for cat in list_cat]), 
                    output='resamp_mmu_vrt')

0

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

# Extract vectorial outputs at the 'commune' level

## Import layers for the whole wallonia

In [61]:
run ../SRC/config_postprocess.py

In [57]:
# Create mapset 
launch_mapset("PERMANENT")

["Location 'WALOUS_31370' already exist",
 "'PERMANENT' mapset already exists in location 'WALOUS_31370'",
 "You are now working in mapset 'WALOUS_31370/PERMANENT'"]

**Communes**

In [58]:
# Import vector layer of the belgian communes
gscript.run_command('v.import', overwrite=True, 
                    epsg='31370', input=data['communes'][1], output=data['communes'][0])

0

In [59]:
# Get list of communes ID
list_com = [code 
            for code 
            in gscript.read_command('v.db.select', flags='c', map=data['communes'][0], columns='cd_munty_r').split('\n') 
            if len(code)>0]

**Cadastral blocks**

In [62]:
# Import vector layer of the belgian communes
gscript.run_command('v.import', overwrite=True, 
                    epsg='3812', input=data['CaPa'][1], output=data['CaPa'][0])

0

## Extract LC proportion by CaPa for each commune

In [56]:
# List of communes ID intersecting MARCHE area
list_com = ['61012','83012','83028','83034','83040','91030','91064','91120']

In [ ]:
# Define region based on the 1m spatial resolution raster LC output
g.region raster=clip_output

In [ ]:
v.to.rast --overwrite input=CaPa output=CaPa use=attr attribute_column=cat label_column=CaPaKey memory=2000

# Extract polygons for each 'commune'

In [ ]:
v.db.addcolumn map=test_overlay@3219 columns="class integer"

In [ ]:
v.db.update map=test_overlay@3219 column=class query_column=a_class where="b_class is null"

In [ ]:
v.db.update map=test_overlay@3219 column=class query_column=a_class where="(a_class is not null) AND (b_class is not null) "

In [ ]:
v.dissolve input=test_overlay@3219 column=class output=test_dissolve